# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import numpy as np, pandas as pd, seaborn as sns, matplotlib.pyplot as plt, scipy
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

Review the code Professor Melnikov used to load and apply a Sentence Transformer model to encode sentences and semantically search for a document.




## **Sentence Encoder as a Language Model**

In the previous video, Professor Melnikov demostrated word embeddings using the `paraphrase-distilroberta-base-v` (1330 MB) model. In this activity, you will use a smaller model, `paraphrase-albert-small-v2` (~50 MB), which encodes any sized text into a 768-dimensional vector. These models, developed by UKP Lab in Germany, are often called Sentence Transformers or Sentence BERT (SBERT) because they extend the popular Google's model BERT. Key differences among models include training corpora, model compression, output vector sizes, and neural network training parameters. It's hard to objectively evaluate (or even define) the model quality, but larger models (in terms of size of training corpora, model architecture and model file bytes) tend to better represent the language on which they were trained. However, newer models can be smaller, faster, and more "precise". For your specific tasks, you would need to experiment with the candidate models to better understand their fit and effectiveness of your task.

Begin by loading the pre-trained language model into a sentence transformer object.

In [ ]:
%time SBERT = SentenceTransformer('paraphrase-albert-small-v2')  # load a pre-trained language model
#  ? SBERT.encode   # to view more help documents about SBERT's encoding method

Next, load 15 famous quotes about language into the `DsQuote` dictionary. You will apply SBERT to find the most relevant quote based on a query phrase.

In [ ]:
DsQuote = dict(  # dictionary of famous quotes about language
  Federico_Fellini          = "A different language is a different vision of life.",
  Ludwig_Wittgenstein       = "The limits of my language mean the limits of my world.",
  Frank_Smith               = "One language sets you in a corridor for life. Two languages open every door along the way.",
  Johann_Wolfgang_von_Goethe= "He who knows no foreign languages knows nothing of his own.",
  Geoffrey_Willans          = "You can never understand one language until you understand at least two.",
  Charlemagne               = "To have another language is to possess a second soul.",
  Karl_Albrecht             = "Change your language and you change your thoughts.",
  Roger_Bacon               = "Knowledge of languages is the doorway to wisdom.",
  Oliver_Wendell_Holmes     = "Language is the blood of the soul into which thoughts run and out of which they grow.",
  Czech_Proverb             = "Learn a new language and get a new soul.",
  Gaston_Bachelard          = "A special kind of beauty exists which is born in language, of language, and for language.",
  Chinese_Proverb           = "Learning is a treasure that will follow its owner everywhere.",
  Marcus_Fabius_Quintilian  = "One should not aim at being possible to understand but at being impossible to misunderstand.",
  Bob_Dylan                 = "A mistake is to commit a misunderstanding.",
  William_Gibson            = "Language is to the mind more than light is to the eye.")

LsQuote = list(DsQuote.values())   # save quotes as a list of strings
print(f'{len(LsQuote)}', LsQuote)

## **Embedding Quotes Into a Vector Space**

Encoding sentences into vectors can be performed by passing each sentence or a list of sentences through the `encode` method of SBERT. This will produce a list of 768-dimensional vectors.

The 15 vectors are packed into a dataframe and colored for readability. The last 15 dimensions are displayed on screen.

In [ ]:
%time LvEmb = SBERT.encode(LsQuote)    # generates a 768-dimensional vector for each sentence

df = pd.DataFrame(LvEmb, index=LsQuote)
cm = sns.light_palette("brown", as_cmap=True)
print(f'df.shape = {df.shape}')
df.iloc[:, -15:].style.background_gradient(cmap=cm, axis=1).set_precision(2)

The rows represent the quotes, while the columns represent the mysterious vector space dimensions. It's not necessary to fully understand the attributes of these columns (or features) to use these to relate columnar values. For example, similar large values indicate greater similarity along that particular dimension. The more similarities, the more similar are the represented quotes. You can look for two sentences that seem similar to you and check whether this is reflected in the values of their coefficients.

## **Computing Similarities Among Quotes**

A more interesting comparison is at the aggregate level. Instead of comparing individual columnar values, you can evaluate correlations (i.e., pairwise measures of linear dependence), which can be easily computed using the `.corr()` method on the transposed dataframe.

The result is the square (and symmetric) matrix of correlation values ranging from -1 to 1 with ones on diagonal, indicating perfect correlation of a sentence to itself.

In [ ]:
InsertNewlines = lambda lst, n=2: '\n'.join([' '.join(lst[i:i + n]) for i in range(0, len(lst), n)]) # n=words to keep together
QuotesWithNewlines = [InsertNewlines(sQuote.split()) for sQuote in LsQuote]

dfSim = pd.DataFrame(df.T.corr().values, index=LsQuote, columns=QuotesWithNewlines)  # Correlation matrix
plt.figure(figsize=[25,4]);
sns.heatmap(dfSim, cbar=False, annot=True, cmap='coolwarm');
plt.xticks(rotation=0);
plt.title('Pearson Correlation Coefficients for Pairs of Sentence Vectors');

Note that row/column labels are now all sentences. Essentially, this eliminated all model coefficient level information.

The correlation matrix highlights similar sentences in either upper or lower triangular. (You can right-click on the image and select **Open Image in New Tab** to view a larger image). More similar sentences will have values close to one (highlighted in pink).

## **Semantic Search for a Quote**

Finally, you can use SBERT to encode a text query `sQuery` into a sentence vector `vQuery`, which can then be used to compute cosine similarity or correlation of `vQuery` with every other quote's vector. These correlations are packed into a dataframe and ordered by highest cosine similarity. 

Try each of the different values for `sQuery` and compare your results. 

In [ ]:
#sQuery = 'talking and thinking'
#sQuery = 'meaning of life'
sQuery = 'language and sight'

vQuery = SBERT.encode(sQuery)  # encode a query string with a 768-dim sentence vector
CosSim = lambda u,v: 1 - scipy.spatial.distance.cosine(u,v) # calc cosine similarity from cosine distance
LsCosSim = [CosSim(vQuery, v) for v in LvEmb]               # cosine similarity for query vector vs each quote vector
dfCosSim = pd.DataFrame(LsCosSim, index=LsQuote, columns=['CosSim']).sort_values('CosSim', ascending=False)
dfCosSim

Note that here you are building a semantic search, not a keyword based search. So, regardless of word morphologies, capitalization or sentence structure, the vectors represent the semantic meaning of each sentence. Hence, the pairwise similarity measures the semantic similarity among sentences and the query phrase.

When searching quotes are semantically similar, a higher cosine similarity is computed. You can verify that the top quote is "most similar" to the query phrase `'language and sight'`.

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**

Now you will practice application of SBERT to document search and comparison. 

As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

## Task 1

Use [`cosine_similarity()`](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.pairwise.cosine_similarity.html) to compute pairwise cosine similarity among SBERT vectors of 15 quotes. The result should be a 15 by 15 dataframe similar to that of pairwise correlations above.

<b>Hint:</b> You can just pass <code>df</code> as an argument to <code>cosine_similarity()</code> and then provide row and column labels as is done above for <code>dfCSim</code> dataframe.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
dfCSim = pd.DataFrame(cosine_similarity(df), index=LsQuote, columns=QuotesWithNewlines)
plt.figure(figsize=[25,4]);
sns.heatmap(dfCSim, cbar=False, annot=True, cmap='coolwarm');
plt.xticks(rotation=0);
plt.title('Cosine Similarities for Pairs of Sentence Vectors');
</pre>
</details> 
</font>

<hr>

## Task 2

Use [`np.corrcoef()`](https://numpy.org/doc/stable/reference/generated/numpy.corrcoef.html) (or any other Pearson correlation function) to compute pairwise correlations among `vQuery` and each of the 15 query vectors. The result should be a 15x1 dataframe similar to `dfCosSim` above.

<b>Hint:</b> Note that <code>np.corrcoef()</code> returns a 2x2 symmetric matrix of correlation coefficients with ones on diagonal. You will need to extract the value in the position [0,1] or the identical value in the position [1,0]. Without such extraction you will receive an error message when trying to treat these matrices as scalar values.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
LsCorr = [np.corrcoef(vQuery, v)[0,1] for v in LvEmb]  # correlation coefficient for query vector vs each quote vector
dfCorr = pd.DataFrame(LsCorr, index=LsQuote, columns=['Corr']).sort_values('Corr', ascending=False)
dfCorr
</pre>
</details> 
</font>

<hr>

## Task 3

Check whether any sentences in `dfCorr` are ordered differently from how they are ordered in `dfCosSim`.

<b>Hint:</b> A simple way to draw row index labels is via <code>dfCorr.index</code> method. You can then Boolean-compare two indices to view where the order is misaligned.

In [ ]:
# check solution here

<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre class="ec">
dfCorr.index==dfCosSim.index
</pre>
</details> 
</font>

<hr>